In [5]:
import numpy as np
import difflib
import pandas as pd

# ------------------------------- Data Preprocessing -----------------------------------

### Phrase -> index

In [6]:
phr_to_ind = dict()

with open('./Datasets/SST1_dataset/dictionary.txt') as f:
    for line in f:
        entry = line.split('|')
        phr_to_ind[entry[0]] = int(entry[1])

keys = phr_to_ind.keys();

print(len(phr_to_ind), phr_to_ind['Good'])

239232 14058


### Getting Index corresponding to sentences

In [7]:
# Without doing the below computation directly load the stored output
sentence_list = []
sentiment = []

with open('./Datasets/SST1_dataset/SentenceWithCorrection.txt') as f:
    for line in f:
        sent = line[:-1]
        sentence_list.append(sent)
        sentiment.append(phr_to_ind[sent])

print(len(sentence_list))

11855


In [8]:
# sentence_list = []
# sentiment = []

# with open('../../Datasets/SST1_dataset/datasetSentences.txt') as f:
#     f.readline()
#     for line in f:
#         entry = line.split('\t')
#         sent = entry[1][:-1]
#         sent = sent.replace('-LRB-', '(')
#         sent = sent.replace('-RRB-', ')')
    
#         if sent in phr_to_ind.keys():
#             sentiment.append(phr_to_ind[sent])
#         else:
#             print('.', end="")
#             keys_subset = [k for k in keys if (k[0] == sent[0])]
#             key = difflib.get_close_matches(sent, keys_subset, n=1);
#             sent = key[0]
#             sentiment.append(phr_to_ind[sent])
            
#         sentence_list.append(sent)
        
# print(len(sentence_list))

# # Written the output in a file
# f = open('../../Datasets/SST1_dataset/SentenceWithCorrection.txt', 'w')
# for sent in sentence_list:
#     f.write(sent + '\n')
# f.close()

### Phrase Index -> Sentiment

In [9]:
ind_to_senti = dict()

with open('./Datasets/SST1_dataset/sentiment_labels.txt') as f:
    f.readline()
    for line in f:
        entry = line.split('|')
        ind_to_senti[int(entry[0])] = float(entry[1])

print(len(ind_to_senti))

239232


### Loading train, test and valid split info

In [10]:
split_ind = []
with open('./Datasets/SST1_dataset/datasetSplit.txt') as f:
    f.readline()
    for line in f:
        entry = line.split(',')
        split_ind.append(int(entry[1]))

print(len(split_ind))

for i in range(len(split_ind)):
    if split_ind[i] == 3:
        split_ind[i] = 1
        
N_train = split_ind.count(1)
N_test = split_ind.count(2)
N_valid = split_ind.count(3)
print (N_train, N_test, N_valid)

11855
9645 2210 0


### Assigning label to sentences

In [11]:
N_sent = len(sentence_list);
N_category = 5

y_label = []

for ind in sentiment:
    val = ind_to_senti[ind]
    if val >= 0.0 and val <= 0.2:
        y_label.append(0);
    elif val > 0.2 and val <= 0.4:
        y_label.append(1)
    elif val > 0.4 and val <= 0.6:
        y_label.append(2)
    elif val > 0.6 and val <= 0.8:
        y_label.append(3)
    else:
        y_label.append(4)

print(y_label.count(0), y_label.count(1), y_label.count(2), y_label.count(3))

# Labels in one-hot encoding
y_train = np.zeros((N_train, N_category), np.uint8)
y_test  = np.zeros((N_test , N_category), np.uint8)
y_valid = np.zeros((N_valid, N_category), np.uint8)

c1,c2,c3 = 0,0,0
for i in range(len(y_label)):
    label = y_label[i]
    if split_ind[i] == 1:
        y_train[c1, label] = 1;  c1 += 1
    elif split_ind[i] == 2:
        y_test [c2, label] = 1;  c2 += 1
    else:
        y_valid[c3, label] = 1;  c3 += 1

1510 3140 2242 3111


### Reducing the size of vocabulary

In [12]:
x_all = []
max_sent_len = -1;
max_wrd_len = -1
wrd_to_ind = dict()

ind_new = 1;
for sent in sentence_list:
    wrds = sent.split()
    vec = []
    for wrd in wrds:
        if wrd not in wrd_to_ind.keys():
            wrd_to_ind[wrd] = ind_new
            ind_new += 1
            
        ind = wrd_to_ind[wrd]
        vec.append(ind)
            
    max_sent_len = max(len(vec), max_sent_len)
    x_all.append(vec)

# Get inverse dictionary
ind_to_wrd = dict((v, k) for k, v in wrd_to_ind.items())
ind_to_wrd[0] = "<PAD/>"

print(len(phr_to_ind), len(wrd_to_ind))

239232 21699


### Create input features

In [13]:
x_train = []
x_test = []
x_valid = []

c1, c2, c3 = 0,0,0
for i in range(len(x_all)):
    vec = x_all[i]
    if split_ind[i] == 1:
        x_train.append(vec)
        c1 += 1
    elif split_ind[i] == 2:
        x_test.append(vec)
        c2 += 1
    else:
        x_valid.append(vec)
        c3 += 1

print(c1, c2, c3)

9645 2210 0


In [20]:
df = pd.DataFrame({'Phrases':sentence_list, 'Label':y_label, 'split_ind':split_ind})
df.to_csv('./Datasets/SST1_dataset/Processed_SST1.tsv', sep='\t', index=False)
df

,Label,Phrases,split_ind
0,3,The Rock is destined to be the 21st Century 's...,1
1,4,The gorgeously elaborate continuation of `` Th...,1
2,2,Effective but too-tepid biopic,2
3,3,If you sometimes like to go to the movies to h...,2
4,4,"Emerges as something rare , an issue movie tha...",2
5,2,The film provides some great insight into the ...,2
6,4,Offers that rare combination of entertainment ...,2
7,3,Perhaps no picture ever made has more literall...,2
8,3,Steers turns in a snappy screenplay that curls...,2
9,3,But he somehow pulls it off .,2


In [21]:
df2 = pd.read_csv('./Datasets/SST1_dataset/Processed_SST1.tsv', sep='\t')
df2

,Label,Phrases,split_ind
0,3,The Rock is destined to be the 21st Century 's...,1
1,4,The gorgeously elaborate continuation of `` Th...,1
2,2,Effective but too-tepid biopic,2
3,3,If you sometimes like to go to the movies to h...,2
4,4,"Emerges as something rare , an issue movie tha...",2
5,2,The film provides some great insight into the ...,2
6,4,Offers that rare combination of entertainment ...,2
7,3,Perhaps no picture ever made has more literall...,2
8,3,Steers turns in a snappy screenplay that curls...,2
9,3,But he somehow pulls it off .,2
